## Import

In [1]:
import keras
import numpy as np
import os
import matplotlib.image
import keras_preprocessing.image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

Using TensorFlow backend.


## Preprocess Images

In [2]:
def preprocessImages():
    for root, dirs, files in os.walk("/jupyter/Classification/assets/gear_images"):
        if(len(dirs)==0):
            p_dir = os.path.join(root).replace('gear_images','p_gear_images')
            if not os.path.exists(p_dir):
                os.makedirs(p_dir)
            print('walking ' + os.path.basename(root))
            for name in files:
                image = keras_preprocessing.image.load_img(os.path.join(root, name), target_size=(128,128))
                image.save(os.path.join(p_dir, name))
    return True

In [3]:
preprocessImages()

walking boots
walking hardshell_jackets
walking axes
walking carabiners
walking helmets
walking harnesses
walking rope
walking crampons
walking gloves
walking tents
walking insulated_jackets
walking pulleys


True

## Training

In [4]:
def loadImages():
    images = []
    labels = []
    for root, dirs, files in os.walk("/jupyter/Classification/assets/p_gear_images"):
        if(len(dirs)==0):
            print('walking ' + os.path.basename(root))
            for name in files:
                labels.append(os.path.basename(root))
                images.append(matplotlib.image.imread(os.path.join(root, name)))
    print('loaded all images')
    x = np.array(images)
    y = np.array(labels)
    dictionary = list(set(labels))
    dictionary.sort()
    for label in dictionary:
        y[y==label]=dictionary.index(label)
    return x, y, dictionary

In [5]:
# loade images and labels
x,y,dictionary = loadImages()
print(dictionary)

walking boots
walking hardshell_jackets
walking axes
walking carabiners
walking helmets
walking harnesses
walking rope
walking crampons
walking gloves
walking tents
walking insulated_jackets
walking pulleys
loaded all images
['axes', 'boots', 'carabiners', 'crampons', 'gloves', 'hardshell_jackets', 'harnesses', 'helmets', 'insulated_jackets', 'pulleys', 'rope', 'tents']


In [6]:
# Prepare data
num_classes = len(dictionary)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (1697, 128, 128, 3)
1697 train samples
425 test samples


In [7]:
# Define model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# Train model
epochs = 10
batch_size = 32

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1697 samples, validate on 425 samples
Epoch 1/10
1697/1697 [==============================] - 66s 39ms/step - loss: 1.8293 - acc: 0.4543 - val_loss: 0.9070 - val_acc: 0.7506
Epoch 2/10
1697/1697 [==============================] - 65s 39ms/step - loss: 0.7893 - acc: 0.7413 - val_loss: 0.6215 - val_acc: 0.8024
Epoch 3/10
1697/1697 [==============================] - 66s 39ms/step - loss: 0.4994 - acc: 0.8427 - val_loss: 0.4189 - val_acc: 0.8776
Epoch 4/10
1697/1697 [==============================] - 65s 39ms/step - loss: 0.3755 - acc: 0.8886 - val_loss: 0.3656 - val_acc: 0.8988
Epoch 5/10
1697/1697 [==============================] - 65s 38ms/step - loss: 0.3032 - acc: 0.9110 - val_loss: 0.3449 - val_acc: 0.8965
Epoch 6/10
1697/1697 [==============================] - 66s 39ms/step - loss: 0.2031 - acc: 0.9393 - val_loss: 0.3263 - val_acc: 0.9059
Epoch 7/10
1697/1697 [==============================] - 68s 40ms/step - loss: 0.1605 - acc: 0.9511 - val_loss: 0.4579 - val_acc: 0.8659
E

In [9]:
# Save model
model.save('/jupyter/Classification/cnn_model_gears.h5')

## Predict using model

In [10]:
# model = load_model('/jupyter/Classification/cnn_model_gears.h5')
# https://shop.epictv.com/sites/default/files/ae42ad29e70ba8ce6b67d3bdb6ab5c6e.jpeg
image = keras_preprocessing.image.load_img('/jupyter/Classification/assets/untrained_image.jpeg', target_size=(128,128))
dictionary = ['axes', 'boots', 'carabiners', 'crampons', 'gloves', 'hardshell_jackets', 'harnesses', 'helmets', 'insulated_jackets', 'pulleys', 'rope', 'tents']
print(dictionary[model.predict(np.array([np.array(image)])).argmax()])

helmets
